In [1]:
!pip freeze

accelerate==0.27.2
aiohttp==3.9.1
aiosignal==1.3.1
annotated-types==0.6.0
annoy==1.17.3
antlr4-python3-runtime==4.9.3
anyio @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_a17a7759g2/croot/anyio_1706220182417/work
appdirs==1.4.4
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1707233003401/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///Users/cbousseau/work/recipes/ci_py311/argon2-cffi-bindings_1677915727169/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_02efro5ps8/croot/async-lru_1699554529181/work
attrs @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_224434dqzl/croot/attrs_1695717839274/work
Babel @ file:///Users/cbousseau/work/recipes/ci_py311/babel_1677920677615/work
backoff==2.2.1
beautifulsoup4 @ file:///private/var/folders/nz/j6p

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
%cd drive/MyDrive/ANLP/NLP-RAG/src-rag
!ls

[Errno 2] No such file or directory: 'drive/MyDrive/ANLP/NLP-RAG/src-rag'
/Users/vashisth/Documents/GitHub/ANLP_projects/NLP-RAG/src-rag
FlanT5.ipynb
LlamaCcp New.ipynb
LlamaCcp.ipynb
Llama_qa_gen_pipeline.ipynb
__pycache__
database.py
evaluation.py
json_csv_embedding.ipynb
setup.txt
src-rag_default_llama.txt
testing_json_embedding.ipynb
webpages_llamaccp_old.txt
webpages_llamaccp_withoutprompt_1shotexample.txt
webpages_llamaccp_withoutprompt_extraction.txt
webpages_llamaccp_withouttemplate.txt


In [4]:
import os
# from dotenv import load_dotenv
# load_dotenv('.env')
# hf_api = os.getenv('HF_API')

In [5]:
pip install transformers==4.38.0

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl

In [7]:
# !pip install unstructured ragatouille
# # reranker
# from ragatouille import RAGPretrainedModel

In [8]:
# # this will take time :c
# !CUDACXX=/usr/local/cuda-12/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

In [9]:
# !pip install ctransformers[cuda]

In [10]:
!pip3 install numpy==1.26.4
import numpy as np
np.__version__

'1.26.4'

In [11]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
import pandas as pd

In [14]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [15]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

In [16]:
# pip list --outdated

# Specify the model/versions, etc.
## also specify the knowledge base file and the reference answer directory

In [17]:
QUESTIONS_FILE = '../data/test/questions.txt' # change when we have all the test ones / for the category wise performance analysis
EMBEDDING_MODEL = "thenlper/gte-base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
# RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)

In [18]:
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    # model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [19]:
# from langchain_community.document_loaders import TextLoader, CSVLoader,JSONLoader
# from database import split_documents, create_db
# import glob
# directory_jsons = '../data/paper_jsons/'
# json_files = glob.glob(os.path.join(directory_jsons, '**', '*.json'), recursive=True)

In [20]:
# # works but if we want to retain the formatting the Jsonloader is better
# loader = DirectoryLoader(directory_jsons, glob="**/*.json",use_multithreading = True, loader_cls=TextLoader)
# docs_json = loader.load()

In [21]:
# !pip install jq

In [22]:
# docs_json = []
# for file_path in json_files:
#   loader = JSONLoader(
#     file_path=file_path,
#     jq_schema='.content',
#     text_content=False,
#     json_lines=True)

# data = loader.load()


In [23]:
# len(docs_json), type(docs_json)

In [24]:
# docs_json_processed = split_documents(
#         chunk_size = 400,
#         chunk_overlap = 50,
#         knowledge_base = docs_json,
#         tokenizer_name=EMBEDDING_MODEL_NAME,
# )

In [25]:
# db_json = create_db(docs_json)

In [26]:
# db_json.save_local("../faiss_index_json")

# Running on processed jsons

In [28]:
from langchain_community.document_loaders import TextLoader, CSVLoader
import glob
from database import split_documents, create_db
directory_csv = '../data/papers_metadata_csv/'
csv_files = glob.glob(os.path.join(directory_csv, '**', '*.csv'), recursive=True)

In [29]:
len(csv_files)

337

In [30]:
loader_csv = DirectoryLoader(directory_csv, glob="**/*.csv",
                             use_multithreading = True,
                             loader_cls=CSVLoader,
                            #  loader_kwargs = csv_args
                             )
docs_csv = loader_csv.load()
docs_csv

[Document(page_content='Question: What is the author ID of Shinji Watanabe?\nAnswer: 1746678\nNotes: ##Title: FINDINGS OF THE IWSLT 2023 EVALUATION CAMPAIGN', metadata={'source': '../data/papers_metadata_csv/f7e995c3cae465963ecaa8c9b4ce8b9b4323a71b.csv', 'row': 0}),
 Document(page_content='Question: What is the H-index of Shinji Watanabe?\nAnswer: 67\nNotes: ##Title: FINDINGS OF THE IWSLT 2023 EVALUATION CAMPAIGN', metadata={'source': '../data/papers_metadata_csv/f7e995c3cae465963ecaa8c9b4ce8b9b4323a71b.csv', 'row': 1}),
 Document(page_content='Question: What is the semantic scholar author name of Shinji Watanabe?\nAnswer: Shinji Watanabe\nNotes: ##Title: FINDINGS OF THE IWSLT 2023 EVALUATION CAMPAIGN', metadata={'source': '../data/papers_metadata_csv/f7e995c3cae465963ecaa8c9b4ce8b9b4323a71b.csv', 'row': 2}),
 Document(page_content='Question: What is the semantic scholar url of Shinji Watanabe?\nAnswer: https://www.semanticscholar.org/author/1746678\nNotes: ##Title: FINDINGS OF THE IWS

In [31]:
db_csv = create_db(docs_csv)

Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning,

In [32]:
db_csv.save_local("../faiss_index_csv")

In [33]:
# author csv

In [34]:
csv_args={
          "delimiter": ",",
          "quotechar": '"',
          'fieldnames': ['Question', 'Answer', 'Document', 'Notes']
      }

# author_csv = []
# for file_path in csv_files:
author_csv = '../data/paper_logs/author.csv'
loader = CSVLoader(
    file_path=author_csv,
    csv_args=csv_args,
)
author_csv = loader.load()
author_csv

[Document(page_content='Question: Question\nAnswer: Answer\nDocument: Document\nNotes: Notes', metadata={'source': '../data/paper_logs/author.csv', 'row': 0}),
 Document(page_content='Question: What is the author ID of Alexander Hauptmann?\nAnswer: 145788702\nDocument: ../data/paper_jsons/A. Hauptmann_145788702.json\nNotes: ', metadata={'source': '../data/paper_logs/author.csv', 'row': 1}),
 Document(page_content='Question: What are the papers written by Alexander Hauptmann?\nAnswer: Zero-Shot and Few-Shot Stance Detection on Varied Topics via Conditional Generation, SPAE: Semantic Pyramid AutoEncoder for Multimodal Generation with Frozen LLMs, STMT: A Spatial-Temporal Mesh Transformer for MoCap-Based Action Recognition, DocumentNet: Bridging the Data Gap in Document Pre-training\nDocument: ../data/paper_jsons/A. Hauptmann_145788702.json\nNotes: ', metadata={'source': '../data/paper_logs/author.csv', 'row': 2}),
 Document(page_content='Question: What is the H-index of Alexander Hauptma

In [35]:
db_author_csv = create_db(author_csv)
db_author_csv.save_local("../faiss_index_authors")

Token indices sequence length is longer than the specified maximum sequence length for this model (846 > 512). Running this sequence through the model will result in indexing errors
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning,

In [36]:
db_combined = db_csv
db_combined.merge_from(db_author_csv)

In [38]:
db_combined.save_local("../faiss_index_author_papers_natural_language")

In [ ]:
# json index saving

In [138]:
db_combined = db_json
db_combined.merge_from(db_author_csv)
db_combined.save_local("faiss_index_author_papers_json")